In [1]:
from tqdm import tqdm
import torch
import torch.nn as nn
from torchvision import utils
from torch.utils import data
from torchvision.datasets import ImageFolder
from torchvision import transforms

In [2]:
train_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.Grayscale(),
    transforms.ToTensor(),
])

In [3]:
train_dataset = ImageFolder('../data/train-cat-rabbit', transform=train_transforms)
train_dataset

Dataset ImageFolder
    Number of datapoints: 1600
    Root location: ../data/train-cat-rabbit
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=bilinear, max_size=None, antialias=None)
               RandomHorizontalFlip(p=0.5)
               RandomRotation(degrees=[-30.0, 30.0], interpolation=nearest, expand=False, fill=0)
               Grayscale(num_output_channels=1)
               ToTensor()
           )

In [41]:
test_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.Grayscale(),
    transforms.ToTensor(),
])
test_dataset = ImageFolder('../data/test-images', transform=test_transforms)
test_dataset

Dataset ImageFolder
    Number of datapoints: 15
    Root location: ../data/test-images
    StandardTransform
Transform: Compose(
               Resize(size=(256, 256), interpolation=bilinear, max_size=None, antialias=None)
               Grayscale(num_output_channels=1)
               ToTensor()
           )

In [42]:
validation_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.Grayscale(),
    transforms.ToTensor(),
])
val_dataset = ImageFolder('../data/val-cat-rabbit/', transform=validation_transforms)
val_dataset

Dataset ImageFolder
    Number of datapoints: 414
    Root location: ../data/val-cat-rabbit/
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=bilinear, max_size=None, antialias=None)
               Grayscale(num_output_channels=1)
               ToTensor()
           )

In [43]:
BATCH_SIZE = 64
train_loader = data.DataLoader(train_dataset,BATCH_SIZE,True)
test_loader = data.DataLoader(test_dataset,BATCH_SIZE)
val_loader = data.DataLoader(val_dataset,BATCH_SIZE,True)

In [7]:
sample_train = train_loader.__iter__().__next__()
sample_train

[tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           ...,
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],
 
 
         [[[0.0000, 0.0000, 0.3569,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.3608,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.3647,  ..., 0.3647, 0.3569, 0.3451],
           ...,
           [0.3176, 0.3451, 0.3647,  ..., 0.3020, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.2353, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.2745, 0.0000, 0.0000]]],
 
 
         [[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000

In [8]:
sample_train[0][:16].shape

torch.Size([16, 1, 256, 256])

In [9]:
grid = utils.make_grid(sample_train[0]).detach().numpy().transpose([1,2,0])
grid.shape

(2066, 2066, 3)

In [10]:
# from skimage.viewer import ImageViewer
# ImageViewer(grid).show()

In [11]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv_1 = nn.Conv2d(1,32,3)
        self.conv_2 = nn.Conv2d(32,64,4)
        self.conv_3 = nn.Conv2d(64, 128, 3)
        self.avg_pool = nn.AvgPool2d(2)
        self.dense_1 = nn.Linear(128*30*30,64)
        self.dense_2 = nn.Linear(64,1)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = nn.functional.relu(self.conv_1(x))
        x = self.avg_pool(x)
        x = self.dropout(x)

        x = nn.functional.relu(self.conv_2(x))
        x = self.avg_pool(x)
        x = self.dropout(x)

        x = nn.functional.relu(self.conv_3(x))
        x = self.avg_pool(x)
        x = self.dropout(x)
        x = nn.Flatten()(x)
        x = nn.functional.relu(self.dense_1(x))

        x = self.dense_2(x)
        return torch.sigmoid(x)

model = ConvNet()
# load_data = torch.load('../models/model_checkpoint.pt')
# model.load_state_dict(load_data['model'])

In [12]:
optimizer = torch.optim.Adam(model.parameters(),0.001, weight_decay=0.00001)
# optimizer.load_state_dict(load_data['optimizers'])
criterion = nn.BCELoss()

In [13]:
def train(model, optimizer, criterion, data_loader, epoch:list[int,int],decay=1e-7,mode='train'):
    pbar = tqdm(data_loader)
    pbar.set_description_str(f'Epoch {epoch[0]}/{epoch[1]} ({mode})')
    running_loss = 0.0
    running_accurate = 0
    batch = None
    for images, labels in pbar:
        labels = labels.view((-1,1)).float()
        if mode == 'train':
            model.train()
            optimizer.zero_grad()
        else:
            model.eval()
        outs = model(images)
        preds = torch.round(outs)
        accurate = (preds == labels).sum()
        running_accurate += accurate
        if batch is None:
            batch = labels.size(0)
        accuracy = accurate / labels.size(0)
        loss = criterion(outs, labels)
        reg_loss = 0.0
        for param in model.parameters():
            reg_loss += torch.norm(param,2) * decay
        loss += reg_loss
        running_loss += loss

        if mode == 'train':
            loss.backward()
            optimizer.step()
        pbar.set_postfix({'loss':loss.item(),'accuracy':accuracy.item()})

    epoch_accuracy = running_accurate.item() / (len(data_loader) * batch)
    print(f'{mode}_loss: {running_loss.item():.4f}, {mode}_accuracy: {epoch_accuracy*100:.4f}%')
    return running_loss.item(), epoch_accuracy

In [14]:
def fit(model, optimizer, criterion, train_loader, validation_loader, epochs, decay=1e-7):
    train_info = {'loss':[],'accuracy':[], 'loss_val':[],'accuracy_val':[]}
    for epoch in range(1, epochs+1):
        loss, accuracy = train(model, optimizer, criterion, train_loader,[epoch, epochs],decay,'train')
        train_info['loss'].append(loss)
        train_info['accuracy'].append(accuracy)
        val_loss, val_accuracy = train(model, optimizer, criterion, validation_loader,[epoch, epochs],decay,'validation')
        train_info['loss_val'].append(val_loss)
        train_info['accuracy_val'].append(val_accuracy)
    return train_info

In [15]:
train_data = fit(model, optimizer, criterion, train_loader, val_loader, 50, decay=0)

Epoch 1/50 (train): 100%|██████████| 25/25 [01:59<00:00,  4.79s/it, loss=0.645, accuracy=0.594]


train_loss: 17.2293, train_accuracy: 59.5000%


Epoch 1/50 (validation): 100%|██████████| 7/7 [00:19<00:00,  2.85s/it, loss=0.619, accuracy=0.633]


validation_loss: 4.2082, validation_accuracy: 61.6071%


Epoch 2/50 (train): 100%|██████████| 25/25 [02:30<00:00,  6.01s/it, loss=0.595, accuracy=0.656]


...

train_loss: 2.3026, train_accuracy: 96.9375%


Epoch 41/50 (validation): 100%|██████████| 7/7 [00:09<00:00,  1.41s/it, loss=0.229, accuracy=0.9]  


validation_loss: 5.3325, validation_accuracy: 75.4464%


Epoch 42/50 (train):  64%|██████▍   | 16/25 [01:18<00:44,  4.93s/it, loss=0.0467, accuracy=0.984]


KeyboardInterrupt: 

In [16]:
torch.save({'model':model.state_dict(),'optimizers':optimizer.state_dict()},'../models/model_checkpoint.pt')

In [45]:
train(model, optimizer, criterion, test_loader,[0,1],0,'test')

Epoch 0/1 (test): 100%|██████████| 1/1 [00:00<00:00,  1.98it/s, loss=0.225, accuracy=0.867]

test_loss: 0.2253, test_accuracy: 86.6667%


(0.22528241574764252, 0.8666666666666667)